

### Model Naming and Result Display

1. **Model Naming**:
   - Ensure you keep the name of your model consistent throughout your project. For instance, in the previous training code, the model was saved as **`my_model.pth`**. Using the same name will make it easier to load and evaluate the model later.

2. **Running the Code**:
   - After ensuring the correct model name, run the code in your notebook. Once executed, you will be able to see the results of the evaluation process, including accuracy and loss metrics. This feedback will help you gauge the model’s performance on thtions to take.

In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision import models
import numpy as np


MODEL_PATH = 'my_model.pth'  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model = models.resnet18(num_classes=2)  
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device) 
model.eval()


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


cap = cv2.VideoCapture(0)


def predict_frame(frame):
    image_tensor = transform(frame)
    image_tensor = image_tensor.unsqueeze(0) 
    image_tensor = image_tensor.to(device) 
    with torch.no_grad():
        output = model(image_tensor)
    _, predicted = torch.max(output, 1)
    return predicted.item()


frame_count = 0
n = 3 

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
        frame_count += 1

        if frame_count % n == 0:
            prediction = predict_frame(frame)

     
            label = "Thumbs Up" if prediction == 0 else "Thumbs Down"
            color = (0, 255, 0) if prediction == 0 else (0, 0, 255)
            cv2.putText(frame, f': {label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        
        cv2.imshow('Live Feed', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    print(f"Error: {e}")
finally:
    cap.release()
    cv2.destroyAllWindows()
